# 15 minutes to QCoDeS

This short introduction is aimed mainly for beginner's. Before you can start with your first code suing QCoDeS make sure you have properly set up the environment for QCoDeS (look at [this  documentation](http://qcodes.github.io/Qcodes/start/index.html#installation)). 

## Introduction 

An experimental setup comprises of many instruments. We call an experimental setup as "station". A station is connected to many instruments or devices. QcoDeS provides a way to interact with all these instruments to help users perform the measurements and store the data in a database. To  interact (read/ write/ trigger etc) with individual instruments we have created a [library of drivers](http://qcodes.github.io/Qcodes/api/generated/qcodes.instrument_drivers.html) for commonly used instruments. These drivers implement the commonly needed functionalities of the instruments. 

An "Instrument" can perform many functions. For example, on an oscilloscope instrument we first set a correct trigger level and then obtain a trace. In QCoDeS lingo, we call "trigger_level" and "trace" as parameters of this `instrument`. An instrument at any moment will have many such parameters which together defines the state of the instrument, hence a parameter can be thought of as a state variable of the instrument. QCoDeS provides method to set values of this parameter (set trigger level) and get the values form them (obtain a trace). This way we can interact with all the needed parameters of an instrument and are ready to set up a measurement. 

QCoDeS also have a similar programmatic structure as explained above. QCoDeS structure comprise of a `Station` class which is bucket of objects from `Instrument` class containing many `Parameters`. The value of these parameters are set and measured during a `Measurement`. The `Measurement` class provides context manager for registering the parameters and providing link between different parameters. 


## Imports

Customarily we import as follows:

In [1]:
import qcodes as qc
from qcodes.tests.instrument_mocks import DummyInstrument

%matplotlib notebook
import numpy.random as rd
import matplotlib.pyplot as plt
from functools import partial
import numpy as np

from time import sleep, monotonic

import qcodes as qc
from qcodes import Station, load_or_create_experiment, \
    initialise_database, Measurement, load_by_run_spec, load_by_guid
from qcodes.tests.instrument_mocks import DummyInstrument
from qcodes.dataset.plotting import plot_dataset
from 



## Logging 

AttributeError: module 'applicationinsights' has no attribute '__version__'

## Station creation  

A station is a collection of all the instruments and devices present in your experiment. It can be thought of as a bucket where you can add your `instruments`, `parameters` and other `components`. Each of these terms have a definite meaning in QCoDeS and is explained in later sections. Once a station is properly configured you can use its instance to access these components. Refer to tutorial on [Station](http://qcodes.github.io/Qcodes/examples/Station.html) for more details.

we start with instantiating instrument class which at the moment does not comprise of any instrument or paramater.

In [ ]:
station = qc.Station()

In [ ]:
station.snapshot()

## Instrument

In [ ]:
# A dummy instrument dac with two parameters ch1 and Ch2
dac = DummyInstrument('dac', gates=['ch1', 'ch2'])

# A dummy instrument dmm with two parameters ch1 and Ch2
dmm = DummyInstrument('dmm', gates=['v1', 'v2'])


### Add instruments into station 

In [ ]:
station.add_component(dac)
station.add_component(dmm)

### Simulating the response of instrument 

You may *ignore this block* during the first read. In actual experiments you will not need to simulate the response.

In [ ]:
# a generator to simulate a physical signal, in this case an exponentially
# decaying signal
def exponential_decay(a: float, b: float):
    """
    Yields a*exp(-b*x) where x is put in
    """
    x = 0
    while True:
        x = yield
        yield a*np.exp(-b*x) + 0.02*a*np.random.randn()
        

# and then a bit of "wiring" to make the dmm "measure"
# the exponential decay

ed = exponential_decay(5, 0.2)
next(ed)

def customgetter(dac):
    val = ed.send(dac.ch1())
    next(ed)
    return val

dmm.v1.get = partial(customgetter, dac)

## Measurement and Database

In [ ]:
# initialise_database()
exp = load_or_create_experiment(experiment_name='dataset_context_manager',
                          sample_name="no sample")

### Where does the QCoDeS save my data?

In [ ]:
qc.config["core"]["db_location"]

## Other important concepts

### QCoDeS Configuration  

In [ ]:
station.snapshot()